## This notebook demonstrates the pipeline of solving the CSG problem using BILP-Q.

**Note:** Make sure the scripts *Utils_Solvers.py* and *Utils_CSG.py* in the root directory of the repository is in the same location of this notebook.

#### Install and  Import Dependencies


In [ ]:
!pip install qiskit
!pip install qiskit_optimization

##### Important: Restart the runtime after installing *pylatexenc*

In [ ]:
!pip install pylatexenc                             # for fetching the diagram of the Quantum circuit used to solve

# Important: Restart the runtime after installing 'pylatexenc'

In [ ]:
!pip install dimod                                   #for runninng Quantum Aneealing approach to solve the CSG problem, you need a D-Wave account
!pip install dwave                                   #for runninng Quantum Aneealing approach to solve the CSG problem, you need a D-Wave account
!pip install dwave-system                            #for runninng Quantum Aneealing approach to solve the CSG problem, you need a D-Wave account

In [6]:
from Utils_Solvers import *
from Utils_CSG import *

##Coalition Structure Generation(CSG) Problem Instance

Consider a game consisiting of 3 agents. The value corresponding to each of the 7 possible coalitions are given as a dictionary with coalition string as key and its coalition value as value.

In [7]:
coalition_values={
    '1':30,
    '2':40,
    '3':25,
    '1,2':70,
    '1,3':60,
    '2,3':65,
    '1,2,3':90
}

coalition_values

{'1': 30, '1,2': 70, '1,2,3': 90, '1,3': 60, '2': 40, '2,3': 65, '3': 25}

## Reduce the CSG problem to a Binary Integer Linaer Programming(BILP) Problem

The BILP problem is described as a cost vector $C$, a binary vector $b$ and a $(n \times (2^n-1))$ binary matrix $S$, with  ~$n$~ agents ~$A=\{a_1,a_2,....a_n\}$~ as rows and ~$\mathcal{P}(A)$~ coalitions as columns(exclude the null set).
$S_{i,j}$ is $1$ if $a_i \in C_j$. The solution $x=\{x_1,x_2,..,x_{2^n-1}\}$ is a $(2^n-1)$ binary string,  which represents a $1$ if a coalition is selected in the solution.\
\
The Binary Integer Linear Programming problem is a constraint optimization problem.
The cost function to maximize and the constraint to be satisfied can be defined using the following formulae:
\begin{equation}
 \text{Maximize}\ \sum_{j=1}^{2^n-1} v(C_j).x_j
\end{equation}
\begin{equation}
\begin{split}
\text{subject to}\ \sum_{j=1}^{2^n-1} S_{i,j}.x_j=1\ \text{for}\ i=1,2,....,n\\
x_j \in \{0,1\}\ \ \text{for}\ j=1,2,.....,(2^n-1)
\end{split}
\end{equation}

In [8]:
c,S,b = convert_to_BILP(coalition_values)       # A function in Utils_CSG.py
print(f'c = {c}\nS = {S}\nb = {b}')


c = [30, 40, 25, 70, 60, 65, 90]
S = [[1, 0, 0, 1, 1, 0, 1], [0, 1, 0, 1, 0, 1, 1], [0, 0, 1, 0, 1, 1, 1]]
b = [1, 1, 1]


## Convert the BILP problem to a Quadratic Unconsrained Binary Optimization(QUBO) problem.

QUBO model is expressed as following
\begin{equation}
\begin{split}
    QUBO: min/max\ y = x^T Q x\\
    x \in \mathbb{R}^{n \times 1}\ and\ Q \in \mathbb{R}^{n \times n}
\end{split}
\end{equation}
where x is again the vector of binary decision variables and Q is a square matrix of constants. We can assume that Q is symmetric or upper-triangular matrix, with out any loss of generality.

Symmetric: for all $i,j \in \{1,2,...,n\}$, except $i=j$ replace $q_{ij}$ with $\frac{(q_{ij}+q_{ji})}{2}$

Upper-Triangular matrix: for all $i,j \in \{1,2,...,n\}$, when $j > i$ replace $q_{ij}$ with $q_{ij}+q_{ji}$ and when $j<i$ replace $q_{ij}$ with $0$.


The CSG problem is a maximization problem. 
The converted BILP to QUBO will be a minimization problem.
The constraints in the BILP formulation of the CSG problem will be subtracted from the cost function with penalty coefficients. These penalty coefficients are vital and has to be of high magnitude as the size of the problem increases.

Specific to this tutorial, the chosen penalty coefficient is $50$, which makes sure the solution generated by QUBO problem is always the same as that of the original CSG problem. The value $50$ will be multiplied by $-1$ and added to the cost function which is same as subrating from the cost function.

The below function will generate the linear and quadratic coefficients as dictionaries which constitutes the Q matrix of the QUBO problem.

In [9]:
qubo_penalty = 50 * -1

linear,quadratic = get_QUBO_coeffs(c,S,b,qubo_penalty)        # A function in Utils_CSG.py



print(f"Linear Coefficients = {linear} \n Quadratic Coefficients = {quadratic}")

Linear Coefficients = {'x_6': -240.0, 'x_3': -170.0, 'x_4': -160.0, 'x_5': -165.0, 'x_0': -80.0, 'x_1': -90.0, 'x_2': -75.0} 
 Quadratic Coefficients = {('x_3', 'x_6'): 200.0, ('x_4', 'x_6'): 200.0, ('x_5', 'x_6'): 200.0, ('x_0', 'x_3'): 100.0, ('x_0', 'x_4'): 100.0, ('x_0', 'x_6'): 100.0, ('x_1', 'x_3'): 100.0, ('x_1', 'x_5'): 100.0, ('x_1', 'x_6'): 100.0, ('x_2', 'x_4'): 100.0, ('x_2', 'x_5'): 100.0, ('x_2', 'x_6'): 100.0, ('x_3', 'x_4'): 100.0, ('x_3', 'x_5'): 100.0, ('x_4', 'x_5'): 100.0}


###Explicit display of **Q** matrix of the QUBO problem

In [10]:
Q = np.zeros([len(linear),len(linear)])

#diagonal elements
for key,value in linear.items():
  Q[int(key.split('_')[1]),int(key.split('_')[1])] = value

#non diagonal elements
for key,value in quadratic.items():
  Q[int(key[0].split('_')[1]),int(key[1].split('_')[1])] = value/2
  Q[int(key[1].split('_')[1]),int(key[0].split('_')[1])] = value/2


Q

array([[ -80.,    0.,    0.,   50.,   50.,    0.,   50.],
       [   0.,  -90.,    0.,   50.,    0.,   50.,   50.],
       [   0.,    0.,  -75.,    0.,   50.,   50.,   50.],
       [  50.,   50.,    0., -170.,   50.,   50.,  100.],
       [  50.,    0.,   50.,   50., -160.,   50.,  100.],
       [   0.,   50.,   50.,   50.,   50., -165.,  100.],
       [  50.,   50.,   50.,  100.,  100.,  100., -240.]])

## Solving QUBO using QAOA

In [11]:
#@title Default title text
def QAOA_optimization(linear, quadratic, n_init=10, p_list=np.arange(1,10), info=''):
    backend = BasicAer.get_backend('qasm_simulator')
    optimizer = COBYLA(maxiter=100, rhobeg=2, tol=1.5)
    qubo = create_QUBO(linear, quadratic)
    op, offset = qubo.to_ising()
    qp = QuadraticProgram()
    qp.from_ising(op, offset, linear=True)
    qaoa_mes = QAOA(optimizer=optimizer, reps=1, quantum_instance=backend, initial_point=[0.,0.])
    qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
    final_qaoa_result = qaoa.solve(qubo)
    optimal_p = 1
    optimal_init = [0.,0.]
    time = final_qaoa_result.min_eigen_solver_result.optimizer_time
    min_sol, min_fval, min_prob, min_rank, _ = results_from_QAOA(final_qaoa_result)
    for p in p_list:
        grid_init = [np.random.normal(1, 1, p * 2) for i in range(n_init)]
        it, min_fval = 0, 0
        for init in grid_init:
            it = it + 1
            qaoa_mes = QAOA(optimizer=optimizer, reps=p, quantum_instance=backend, initial_point=init)
            qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
            qaoa_result = qaoa.solve(qubo)
            _, fval, _, rank, _ = results_from_QAOA(qaoa_result)
            if (min_fval>fval) and (rank<min_rank):
                min_fval = fval
                min_rank = rank
                optimal_p = p
                optimal_init = init
                final_qaoa_result = qaoa_result
                time = final_qaoa_result.min_eigen_solver_result.optimizer_time
    qaoa_mes.get_optimal_circuit().draw('mpl', filename='circuit.png')
    return final_qaoa_result, optimal_p, optimal_init, time

In [12]:
qaoa_result, p, init, _ = QAOA_optimization(linear, quadratic, n_init=1, p_list=np.arange(1,2))      # A function in Utils_Solvers.py

solution, fval, prob, rank, time_run = results_from_QAOA(qaoa_result)

solution

array([0., 1., 0., 0., 1., 0., 0.])

## Visualize the quantum circuit used to solve the CSG problem:

In [ ]:
#@title Default title text
from IPython import display
display.Image("./circuit.png")

## Decode the solution

The solution from the QAOA will be an array of binary digiits.
The following piece of code will convert the solution of QUBO to the solution of the original CSG problem, i.e., the Optimal Coaliton Structure

In [8]:
def decode(solution,coalition_values):
  """
  Convert the solution binary string into a coalition sructure(a list of coalitions)
  """
  output = []
  for index,element in enumerate(solution):
    if int(element)!=0:
      output.append(set(list(coalition_values)[index].split(',')))
  return output


decode(solution, coalition_values)

[{'2'}, {'1', '3'}]